<a href="https://colab.research.google.com/github/jaealways/components_on_term_structure/blob/main/term_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data

기간구조 데이터에서 유의미한 정보를 추출하려면, 먼저 데이터에 대해 살펴볼 필요가 있습니다.

In [ ]:
import matplotlib
# matplotlib.use('TkAgg')
import seaborn as sns
sns.set(style="ticks", color_codes=True)
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
#mpl.rcParams['figure.dpi'] = 200

In [ ]:
from IPython.core.display import HTML
HTML("<style>.container {width:98% !important; }</style>")
plt.rcParams['axes.facecolor'] = 'white'
import seaborn as sns
plt.rcParams['axes.facecolor'] = 'white'
sns.set_palette("Set2")

데이터를 호출하는 fredapi를 설치합니다.

In [ ]:
pip install fredapi

In [ ]:
from fredapi import Fred
fred = Fred(api_key='cd9a27c9afcd4ee82ec0be135fb8b223')

# get data
startDate = '2019-09-01'
endDate = '2020-09-01'
df = []
ids = ['DGS{}'.format(i) for i in [1,2,5,7,10,20,30]]
for s in ids:
    df.append(fred.get_series(s, observation_start=startDate,observation_end=endDate)/100)

df = pd.concat(df,axis=1)
df.columns = ids
df = df.dropna()

시간이 지남에 따라 기간구조가 어떤 모습을 보이는지 시각화합니다.

In [ ]:
# curve dynamic
fig,(ax,ax2)=plt.subplots(nrows=2,ncols=1,figsize=(10,5*2))
df.plot(grid=True, title='US Treasury Constant Maturity Rate', ax=ax)
df.index.intersection([datetime(2019,9,1),datetime(2019,3,31),datetime(2020,9,1)])
x = df.loc[df.index.intersection([datetime(2019,9,1),datetime(2019,3,31),datetime(2020,9,1)])]
x.index = [t.date() for t in x.index]
ax2.plot(x.T.index,x.T,marker='*')
ax2.legend(x.index)
ax2.grid(True)
ax2.set_title('Term structure on different dates')
fig.tight_layout()

## 1.1 Normality Test


PCA는 인풋 데이터가 정규분포를 따른다는 가정하에, 그 논리를 전개하기 때문에 기간구조 데이터가 정규분포를 따르는지 검증하는 절차가 필요합니다.




In [ ]:
hist = df.hist(bins=30, sharex=True)

시각화를 진행했을 때, 대부분의 데이터가 multimodal 분포를 따르는 것으로 확인되었는데, 보다 정확한 판별을 위해 정규성 검정을 진행합니다.

### 1.1.1 Q-Q Plot

Q-Q plot을 통해 살펴보면 다음과 같습니다.

In [ ]:
from scipy.stats import probplot
import matplotlib.pyplot as plt

f, axes = plt.subplots(7, 2, figsize=(12,30))
for i in range(7):
    axes[i][0].boxplot(df.iloc[:,i])
    probplot(df.iloc[:,i], plot=axes[i][1])
plt.axis("equal")
plt.show()

### 1.1.2 Shapiro-Test

데이터가 정규성을 만족한다는 귀무가설을 통해, p-value를 반환해서 정규성을 판단하고자 합니다.

In [ ]:
from scipy.stats import shapiro

list_shapiro_test=[]
for i in range(7):
    list_shapiro_test.append(shapiro(df.iloc[:,i]))
list_shapiro_test

모든 데이터의 p-value가 유의미하게 낮게 나와 기각역에 속한다고 볼 수 있습니다. 즉 정규성을 만족하지 못합니다.

# 2. PCA

Principal Component Analysis로 기간구조를 분해하기 위해 [Aleksei Malinovskii, "Term Structure Analysis using PCA", 2022](https://fam.tuwien.ac.at/~sgerhold/pub_files/sem21/s_malinovskii.pdf) 논문을 참고했습니다. 요약본은 제가 작성한 [아티클](https://github.com/jaealways/jaealways.github.io/blob/master/_posts/2023-08-11-Term-Structure-Analysis-using-PCA.md)을 참고하시기 바랍니다.

## 2.1 Principal Components Modification

위의 코드는 논문의 저자가 pca 오픈소스를 그대로 차용하지 않고, term structure 계산에 맞게 변형한 부분입니다.</br></br>
대표적으로 위의 **논문** 3.4.2의 loading matrix V에서 pc1과 pc2의 값을 변형시켜주는 것이 있습니다. code에선 self.adjust_sign에서 이를 판별합니다.

PC1의 경우 항상 양의 load를 갖다보니, np.sign 메소드를 통해 항상 양의 값을 유지하도록 로직을 변경했습니다.
PC2의 경우 "The loading increases from a negative value at the short end to a positive value at
the long end." 부분에 의해 -np.sign을 곱해준 것 같은데, 아직 정확히 이해하진 못했습니다.



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

class PCABase(object):
    def __init__(self, X, adjust_sign=True):
        self.X = X
        self.n_features = X.shape[1]
        self.dates = X.index
        self.Xc = self.X - self.X.mean() # centered
        self.pc_names = lambda n: ['PC' + str(i) for i in np.arange(1, n +1)]
        self.adjust_sign = adjust_sign


    def pca(self, n_pc=None):
        '''
        fit pca model
        n_pc: number of pcs to fit, take total feature numbers if not
        ,→ specified
        '''
        if n_pc:
            model = PCA(n_components=n_pc).fit(self.Xc)
        else:
            model = PCA().fit(self.Xc)
        return model

    def cps(self):
        '''
        loading matrix => principal axes in feature space
        '''
        cps = self.pca().components_.T
        cps = self.to_df_pc(cps, is_loading=True)
        if self.adjust_sign:
            cps.loc[:, 'PC1'] = np.sign(cps.loc[:, 'PC1'].values[0]) * cps.loc[:, 'PC1']
            cps.loc[:, 'PC2'] = -np.sign(cps.loc[:, 'PC2'].values[0]) * cps.loc[:, 'PC2']
        return cps

    def cumsum_expvar_ratio(self):
        var_exp = self.pca().explained_variance_ratio_
        var_exp_cumsum = np.cumsum(var_exp)
        return var_exp, var_exp_cumsum

    def scores(self):
        '''
        PC scores:
        '''
        scores = self.pca().transform(self.Xc)
        scores = self.to_df_pc(scores)
        if self.adjust_sign:
            cps = self.cps()
        scores.loc[:, 'PC1'] = np.sign(cps.loc[:, 'PC1'].values[0]) * scores.loc[:, 'PC1']
        scores.loc[:, 'PC2'] = -np.sign(cps.loc[:, 'PC2'].values[0]) * scores.loc[:, 'PC2']
        return scores

    def scores2(self):
        '''
        equivalent to the sklearn transform function
        '''
        scores = self.Xc.dot(self.cps())
        scores = self.to_df_pc(scores)

        if self.adjust_sign:
            cps = self.cps()
            scores.loc[:, 'PC1'] = np.sign(cps.loc[:, 'PC1'].values[0]) * scores.loc[:, 'PC1']
            scores.loc[:, 'PC2'] = -np.sign(cps.loc[:, 'PC2'].values[0]) * scores.loc[:, 'PC2']
        return scores

    def x_projected(self, p, centered=False):
        xp = self.scores().iloc[:, 0:p].dot(self.cps().T.iloc[0:p, :])
        if not centered:
            xp = xp + self.X.mean()
        return xp

    def residuals(self, p):
        residuals = self.X - self.x_projected(p, centered=False)
        return residuals

    def covX(self):
        return self.X.cov()

    def eigenv(self):
        eig_vals, eig_vecs = np.linalg.eig(self.covX())
        return eig_vals, eig_vecs

    def to_df_pc(self, data, is_loading=False):
        cols = self.pc_names(self.n_features)
        idx = self.X.columns if is_loading else self.dates
        return pd.DataFrame(data, columns=cols, index=idx)

## 2.2 PCA experiment

In [ ]:
# contruct pca object
pcab = PCABase(df)

# loading matrix (direction may change but doesn't matter)
V = pd.DataFrame(pcab.pca().components_,index=pcab.pc_names(pcab.n_features),columns=ids)
V.T.iloc[:,0:3].plot(figsize=(10,5),kind='bar')
pcab.cps()

# resconstruction and residuals
r = 'DGS10'
fig,(ax,ax2)=plt.subplots(figsize=(8,4*2),ncols=1,nrows=2)
ax.plot(pcab.x_projected(1)[r])
ax.plot(pcab.x_projected(2)[r])
ax.plot(pcab.x_projected(3)[r])
ax.legend(['Reconstructed (PC1)','Reconstructed (PC1,PC2)','Reconstructed (PC1,PC2,PC3)'])
ax.set_title('{} reconstructed by PCs'.format(r))

ax2.plot(pcab.residuals(1)[r])
ax2.plot(pcab.residuals(2)[r])
ax2.plot(pcab.residuals(3)[r])
ax2.legend(['Residual (PC1)','Residual (PC1,PC2)','Residual (PC1,PC2,PC3)'])
ax2.set_title('{} residuals from reconstructed by PCs'.format(r))
fig.tight_layout()

In [ ]:
# resconstruction and residuals
r = 'DGS30'
fig,(ax,ax2)=plt.subplots(figsize=(8,4*2),ncols=1,nrows=2)
ax.plot(pcab.x_projected(1)[r])
ax.plot(pcab.x_projected(2)[r])
ax.plot(pcab.x_projected(3)[r])
ax.legend(['Reconstructed (PC1)','Reconstructed (PC1,PC2)','Reconstructed (PC1,PC2,PC3)'])
ax.set_title('{} reconstructed by PCs'.format(r))

ax2.plot(pcab.residuals(1)[r])
ax2.plot(pcab.residuals(2)[r])
ax2.plot(pcab.residuals(3)[r])
ax2.legend(['Residual (PC1)','Residual (PC1,PC2)','Residual (PC1,PC2,PC3)'])
ax2.set_title('{} residuals from reconstructed by PCs'.format(r))
fig.tight_layout()

In [ ]:
# PC Scores
fig,(ax1,ax2,ax3)=plt.subplots(nrows=3,ncols=1,figsize=(8,3*3))
l1=ax1.plot(pcab.scores()['PC1'])
ax12 = ax1.twinx()
l2=ax12.plot(pcab.X['DGS10'],color='orange')
ax1.tick_params('x',rotation=30)
ax1.legend(l1+l2,['PC1 score','DGS10'])
ax1.set_ylabel('score')
ax12.set_ylabel('DGS10')

l1=ax2.plot(pcab.scores()['PC2'])
ax22 = ax2.twinx()
l2=ax22.plot(pcab.X['DGS10']-pcab.X['DGS2'],color='orange')
ax2.tick_params('x',rotation=30)
ax2.legend(l1+l2,['PC2 score','DGS10-DGS2'],loc='best')
ax2.set_ylabel('score')
ax22.set_ylabel('DGS10-DGS2')

l1=ax3.plot(pcab.scores()['PC3'])
ax32 = ax3.twinx()
l2=ax32.plot(pcab.X['DGS30']-2*pcab.X['DGS10']+pcab.X['DGS5'],color='orange')
ax3.tick_params('x',rotation=30)
ax3.legend(l1+l2,['PC3 score','DGS30-2DGS10+DGS5'],loc='best')
ax3.set_ylabel('score')
ax32.set_ylabel('DGS30-2DGS10+DGS5')
fig.tight_layout()

In [ ]:
# PC
fig,ax=plt.subplots(figsize=(10,5))
l1=pcab.scores().iloc[:,0:3].plot(ax=ax)
ax.grid(True)
ax.set_title('Evolution of PCA Factors 1,2,3')

# PCA-explained variance ratio

fig,(ax,ax2) = plt.subplots(figsize=(6*2, 4),ncols=2,nrows=1)
ax.bar(range(pcab.n_features), pcab.cumsum_expvar_ratio()[0], alpha=0.5, align='center')
ax.set_ylabel('Explained variance percentage')
ax.set_xlabel('Principal components')
ax.set_title('Individual PC explained variance percentage')

ax2.bar(range(pcab.n_features), pcab.cumsum_expvar_ratio()[1], alpha=0.5, align='center')
ax2.set_ylabel('Explained variance percentage')
ax2.set_xlabel('Principal components')
ax2.set_title('Cumulative PC explained variance percentage')

fig.tight_layout()

# 3. SVD

# 4.ICA

ICA를 금융데이터에 적용하기 위해 [A First Application of Independent Component Analysis to Extracting
Structure from Stock Returns](https://archive.nyu.edu/bitstream/2451/14180/1/Is-97-22.pdf) 논문을 참조했습니다. 요약본은 제가 쓴 [다음 아티클](https://github.com/jaealways/jaealways.github.io/blob/master/_posts/2023-08-15-A-First-Application-of-Independent-Component-Analysis-to-Extracting-Structure-from-Stock-Returns.md)을 참고하시기 바랍니다.

In [ ]:
from sklearn.decomposition import FastICA
ICA = FastICA(n_components=20, random_state=0)
X_transformed = ICA.fit_transform(df)
A_ = ICA.mixing_.T

plt.figure(figsize = (12, 2))
plt.plot(df.iloc[:,0], label="data")
plt.plot(np.dot(X_transformed[:,0], A_), label="reconstruct")
plt.legend()